In [3]:
import requests
from bs4 import BeautifulSoup 
import json
# do_sleep()
import time
import random
# 時間轉換
from datetime import datetime
# def download_image()
import urllib.request
import os

def set_session_headers(session):
    session.headers = {
        'User-Agent': 'Instagram 200.0.0.18.255 (iPhone; iOS 15_0; en_US; en-US; scale=2.00; 750x1334) AppleWebKit/420+',
        'Accept-Language': 'en-US',
    }
    
def find_user_id(session, username):
    url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={username}'
    response = session.get(url)
    do_sleep()
    data = response.json()
    return data['data']['user']['id']

def download_media(url, path, name, datatype):
    if not os.path.exists(path):
        os.makedirs(path)
    file_format = "01"
    # Convert back to a string and use zfill to pad with zeros
    fullname = str(name).zfill(len(file_format))+f".{datatype}"
    if not os.path.exists(path+fullname):
        urllib.request.urlretrieve(url,path+fullname)
    
def find_image_video_url(item, image_url, video_url):
    curr_width = 0
    curr_url = ''
    # media['image_versions2']['candidates'] 貼文內該圖片的解析度候選，找出解析度最高的圖片連結
    for candidate in item['image_versions2']['candidates']:
        if candidate['width']>curr_width:
            curr_width = candidate['width']
            curr_url = candidate['url']
    # 將解析度最高之圖片連結儲存
    image_url.append(curr_url)

    if 'video_versions' in item.keys():
        curr_width = 0
        curr_url = ''
        # media['video_versions'] 貼文內該影片的解析度候選，找出解析度最高的圖片連結
        for candidate in item['video_versions']:
            if candidate['width']>curr_width:
                curr_width = candidate['width']
                curr_url = candidate['url']
        # 將解析度最高之影片連結儲存
        video_url.append(curr_url)
            
def post_data(session, user_id, path, count='', max_id=''):
    url = f'https://www.instagram.com/api/v1/feed/user/{user_id}/?count={count}&max_id={max_id}'
    response = session.get(url)
    data = response.json()

    for item in data['items']:
        # post_time or data['items'][0]['caption']['created_at_utc'] 發文時間(為 timestamp)，
        # 轉成 int 後還原為 YYYY-MM-DD hh:mm:ss後，
        # 轉成 string 將 : 換為空字元，用來做儲存路徑
        if item['caption']==None:
            post_time = str(datetime.utcfromtimestamp(int(item['device_timestamp'])//1e6)).replace(':', '').replace(' ', '_')
        else:
            post_time = str(datetime.fromtimestamp(int(item['caption']['created_at_utc']))).replace(':', '').replace(' ', '_')
        
        # find image and video link
        image_url = []
        video_url = []
        # 多張圖片
        if 'carousel_media' in item.keys():
            # data['items'][]['carousel_media'] 貼文內圖片影片
            for media in item['carousel_media']:
                find_image_video_url(media, image_url, video_url)
        # 單張圖片
        else:
            find_image_video_url(item, image_url, video_url)
        print(post_time, 'image num:',len(image_url), 'video num:',len(video_url))
        
        # download_media
        for count, url in enumerate(image_url):
            do_sleep()
            download_media(url, path+post_time+'/', count+1, 'jpg')
        for count, url in enumerate(video_url):
            do_sleep()
            download_media(url, path+post_time+'/', count+1, 'mp4')
            
    if 'next_max_id' in data.keys():
        if data['next_max_id']:
            do_sleep()
            load_post_data(session, user_id, path, data['num_results'], data['next_max_id'])
    else:
        # data end
        return
        
def do_sleep():
    time.sleep(min(random.expovariate(0.9), 30.0))
    
if __name__ == "__main__":
    username = ""
    path = f'../save_path/{username}/'
    # create requests session
    session = requests.Session()
    # set session headers
    set_session_headers(session)
    
    # find user id
    user_id = find_user_id(session, username)
    # print(user_id)
    do_sleep()
    
    # load_post_date
    post_data(session, user_id, path)
    # print('end')

47598283663
2023-12-12_173145 image num: 9 video num: 0
2023-12-09_200306 image num: 9 video num: 0
2023-12-04_190024 image num: 10 video num: 0
2023-12-03_171014 image num: 9 video num: 0
2023-11-28_230015 image num: 10 video num: 0
2023-11-24_165951 image num: 10 video num: 2
2023-11-17_170006 image num: 6 video num: 0
2023-11-09_170024 image num: 7 video num: 2
2023-11-08_150040 image num: 10 video num: 1
2023-11-06_170012 image num: 1 video num: 0
2023-10-31_190210 image num: 1 video num: 0
2023-10-29_170104 image num: 9 video num: 0
2023-10-26_190025 image num: 9 video num: 0
2023-10-25_170013 image num: 1 video num: 0
2023-10-21_214609 image num: 1 video num: 0
2023-10-20_120006 image num: 8 video num: 0
2023-10-17_124906 image num: 1 video num: 0
2023-10-16_203006 image num: 8 video num: 0
2023-10-13_110725 image num: 10 video num: 0
2023-10-07_120020 image num: 1 video num: 0
2023-10-04_173120 image num: 9 video num: 2
2023-10-04_170011 image num: 9 video num: 0
2023-10-01_1701

2021-12-11_201921 image num: 6 video num: 0
2021-12-09_170040 image num: 4 video num: 0
2021-12-06_121242 image num: 8 video num: 0
2021-12-05_001749 image num: 4 video num: 0
2021-12-04_120958 image num: 9 video num: 0
2021-11-28_111853 image num: 4 video num: 0
1975-03-11_215534 image num: 3 video num: 0
2021-11-26_200012 image num: 5 video num: 0
2021-11-26_125045 image num: 3 video num: 0
2021-11-22_050552 image num: 1 video num: 0
2021-11-21_130003 image num: 4 video num: 0
2021-11-15_110036 image num: 3 video num: 0
2021-11-13_170006 image num: 4 video num: 0
2021-11-12_154013 image num: 7 video num: 2
2021-11-06_130133 image num: 6 video num: 0
2021-11-05_183010 image num: 5 video num: 0
2021-11-02_115013 image num: 1 video num: 0
2021-10-31_120004 image num: 5 video num: 0
2021-10-30_142747 image num: 3 video num: 0
2021-10-24_123045 image num: 10 video num: 0
2021-10-22_230110 image num: 3 video num: 0
2021-10-22_131928 image num: 4 video num: 0
2021-10-15_035956 image num: 5 